<a href="https://colab.research.google.com/github/yz2873/Unsupervised_Learning/blob/master/Pinecore_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Diving into Pinecone**

In [ ]:
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.0/276.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.286
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## **Load The API Key**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv('/content/drive/My Drive/.env', override=True)

True

In [ ]:
import pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

In [ ]:
pinecone.info.version()

VersionResponse(server='2.0.11', client='2.2.2')

## **Pinecone Indexes**

In [ ]:
pinecone.list_indexes()

['langchain-pinecone']

In [ ]:
index_name = 'langchain-pinecone'
if index_name not in pinecone.list_indexes():
  print(f'Create Index: {index_name}...')
  pinecone.create_index(index_name, dimension=1536, metric='cosine', pods=1, pod_type='p1.x2')
  print('Done')
else:
  print(f'Index {index_name} already exists!')

Create Index: langchain-pinecone...
Done


In [ ]:
index_name = 'langchain-pinecone'
pinecone.describe_index(index_name)

IndexDescription(name='langchain-pinecone', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [ ]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
pinecone.list_indexes()

['langchain-pinecone']

## **Generate a random index and add it into the pinecone.**

In [ ]:
import random

In [ ]:
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')
ids

['a', 'b', 'c', 'd', 'e']

In [ ]:
#Upsert a vectors
index.upsert(vectors=zip(ids, vectors))

{'upserted_count': 5}

In [ ]:
#Update an element
#index.update(vectors=['c', [0.3]*1536])

In [ ]:
#index.fetch(ids=['c', 'd'])

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 5}},
 'total_vector_count': 5}

In [ ]:
# Delete columns 'b' and 'c'
index.delete(ids=['b', 'c'])

{}

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [ ]:
#Remove all the indexes
index.delete(delete_all=True)

{}

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [ ]:
queries = [[random.random() for _ in range(1536)] for v in range(1)]

In [ ]:
index.query(
    queries=queries,
    top_k=3,
    include_values=False
)

{'results': [{'matches': [{'id': 'b', 'score': 0.753049731, 'values': []},
                          {'id': 'e', 'score': 0.745641649, 'values': []},
                          {'id': 'd', 'score': 0.745435059, 'values': []}],
              'namespace': ''}]}

In [ ]:
queries = [[random.random() for _ in range(1536)] for v in range(2)]

In [ ]:
index.query(
    queries=queries,
    top_k=3,
    include_values=False
)

{'results': [{'matches': [{'id': 'a', 'score': 0.757970393, 'values': []},
                          {'id': 'd', 'score': 0.757400632, 'values': []},
                          {'id': 'e', 'score': 0.756070912, 'values': []}],
              'namespace': ''},
             {'matches': [{'id': 'd', 'score': 0.752501726, 'values': []},
                          {'id': 'b', 'score': 0.750305772, 'values': []},
                          {'id': 'e', 'score': 0.747451305, 'values': []}],
              'namespace': ''}]}